In [48]:

import pandas as pd 
import json 

# data_path_tamplate = 'Result/Mixtral_template_RULE_{key}_Mixtral_8bit.jsonl'
# data_path_tamplate = 'Result/Alpaca_template_RULE_{key}_Llama-70B-8bit.jsonl'
data_path_tamplate = 'Result/gpt_results/result_{key}_gpt-4-1106-preview.csv'
path_key_list = ['mainq', 'subq_all']

subq_all = pd.read_json('../Data/RULE/RULE_subq_all.jsonl', lines=True)

def extract_first_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()]))-1

def extract_last_num_in_string(string: str):
    result = ''.join([x for x in string if x.isdigit()])
    if result == '':
        return -1
    else: 
        return int(''.join([x for x in string if x.isdigit()])[-1])-1
    
def correctness_matching(subq_all: pd.DataFrame, subq_id: str):
    if subq_id in subq_all['id_string'].values:
        return subq_all[subq_all['id_string']==subq_id]['main_option_correctness'].values[0]
    else: 
        return None

def extract_mainq_id_string(subq_id: str):
    return subq_id.split('_sub')[0]

mainq_result_path = data_path_tamplate.format(key='mainq')
if 'jsonl' in mainq_result_path:
    mainq_result = pd.read_json(mainq_result_path, lines=True)
elif 'csv' in mainq_result_path:
    mainq_result = pd.read_csv(mainq_result_path)

subq_result_path = data_path_tamplate.format(key='subq_all')
if 'jsonl' in mainq_result_path:
    subq_result = pd.read_json(subq_result_path, lines=True)
elif 'csv' in mainq_result_path:
    subq_result = pd.read_csv(subq_result_path) 
# subq_result = pd.read_json(subq_result_path, lines=True)

if 'jsonl' in mainq_result_path:
    mainq_result['prediction'] = mainq_result['inference'].apply(extract_last_num_in_string)
    correct = 0
    for i in range(len(mainq_result)):
        if mainq_result.label[i] == mainq_result.prediction[i]:
            correct += 1
    print(f'mainq_result, accuracy: {correct/len(mainq_result)}')

    # data['main_option_correctness'] 열 만들기
    subq_result['main_option_correctness'] = subq_result['id_string'].apply(lambda x: correctness_matching(subq_all, x))
    subq_result['prediction'] = subq_result['inference'].apply(extract_last_num_in_string)
    subq_result['mainq_id'] = subq_result['id_string'].apply(extract_mainq_id_string)
    correct = 0
    for i in range(len(subq_result)):
        if subq_result.label[i] == subq_result.prediction[i]:
            correct += 1
    print(f'subq_result, accuracy: {correct/len(subq_result)}')

elif 'csv' in mainq_result_path:
    print(f'mainq_result, accuracy: {len(mainq_result[mainq_result.is_correct == True])/len(mainq_result)}')
    print(f'subq_result, accuracy: {len(subq_result[subq_result.is_correct == True])/len(subq_result)}')
    
    correct = 0
    for i in range(len(mainq_result)):
        # mainq_result가 맞았을 때
        if mainq_result.is_correct[i]:
            # mainq_result.id_string[i]와 같은 subq_result.mainq_id 가진 행을 찾아서 모두 정답을 확인
            subq_result_sub = subq_result[subq_result['main_id']==mainq_result.id[i]]
            
            if True in subq_result_sub.is_correct.values:
                if subq_result_sub.is_correct.value_counts()[True] == len(subq_result_sub):
                    correct += 1
                
    print(f'Consistency: {correct/len(mainq_result)}')

mainq_result, accuracy: 0.8748674443266172
subq_result, accuracy: 0.7555777555777555
Consistency: 0.37751855779427357


In [43]:
subq_result_sub.is_correct.value_counts()[True]

3

In [49]:
# data에서 main_option_correctness 값이 True인 것만 추출하기 
data_true = subq_result[subq_result['main_option_correctness']==True]
correct = 0
for i in range(len(data_true)):
    if data_true.label.values[i] == data_true.prediction.values[i]:
        correct += 1
print(f'data_true, accuracy: {correct/len(data_true)}')

# data에서 main_option_correctness 값이 False인 것만 추출하기
data_false = subq_result[subq_result['main_option_correctness']==False]
correct = 0
for i in range(len(data_false)):
    if data_false.label.values[i] == data_false.prediction.values[i]:
        correct += 1
print(f'data_false, accuracy: {correct/len(data_false)}')

correct = 0
for i in range(len(mainq_result)):
    # mainq_result가 맞았을 때
    if mainq_result.label[i] == mainq_result.prediction[i]:
        # mainq_result.id_string[i]와 같은 subq_result.mainq_id 가진 행을 찾아서 모두 정답을 확인
        subq_result_sub = subq_result[subq_result['mainq_id']==mainq_result.id_string[i]]
        
        correct_ = True
        for j in range(len(subq_result_sub)):
            if subq_result_sub.label.values[j] != subq_result_sub.prediction.values[j]:
                correct_ = False
                break
        if correct_:
            correct += 1

print(f'Consistency: {correct/len(mainq_result)}')

AttributeError: 'DataFrame' object has no attribute 'prediction'

In [2]:
# 아래의 각 값에 가짜 값을 부여함 
inferenced = []
context = 'context'
question = 'question'
answers = ['answer1', 'answer2', 'answer3', 'answer4']
label = 0
id_string = 'id_string'
input_string = 'input_string'
inference = 'inference'
main_option_correctness = True

for i in range(3):
    inferenced.append({
                "context": context,
                "question": question,
                "answers": answers,
                "label": label,
                "id_string": id_string,
                "input_string": input_string,
                "inference": inference,
                "main_option_correctness": main_option_correctness,
            })


In [ ]:
import os 

output_filepath = os.path.join(output_path, f'{output_key}.jsonl')
if output_key == "":
    output_key = data_path.replace(".jsonl", "_inference.jsonl")
with open(output_filepath, "w") as f:
    for x in inferenced:
        f.write(json.dumps(x) + "\n")

In [3]:
inferenced

[{'context': 'context',
  'question': 'question',
  'answers': ['answer1', 'answer2', 'answer3', 'answer4'],
  'label': 0,
  'id_string': 'id_string',
  'input_string': 'input_string',
  'inference': 'inference',
  'main_option_correctness': True},
 {'context': 'context',
  'question': 'question',
  'answers': ['answer1', 'answer2', 'answer3', 'answer4'],
  'label': 0,
  'id_string': 'id_string',
  'input_string': 'input_string',
  'inference': 'inference',
  'main_option_correctness': True},
 {'context': 'context',
  'question': 'question',
  'answers': ['answer1', 'answer2', 'answer3', 'answer4'],
  'label': 0,
  'id_string': 'id_string',
  'input_string': 'input_string',
  'inference': 'inference',
  'main_option_correctness': True}]

In [8]:
import json 
import os 
result_files = [f'Result/Mixtral_template_RULE_mainq_Mixtral_4bit_{x}.jsonl' for x in range(0, 3)]


all_inferenced = []
for rank in range(3):
    rank_output_filepath = result_files[rank]
    with open(rank_output_filepath, "r") as f:
        for line in f:
            all_inferenced.append(json.loads(line))

# 최종 파일 저장
final_output_filepath = os.path.join('Result/Mixtral_template_RULE_mainq_Mixtral_4bit_final.jsonl')
with open(final_output_filepath, "w") as f:
    for x in all_inferenced:
        f.write(json.dumps(x) + "\n")

In [31]:
subq_all = pd.read_json('../Data/RULE/RULE_subq_all.jsonl', lines=True)
subq_all

,main_id,context,question,label,main_question,main_option_correctness,id_string,answers
0,train_1,Patient: Pharmacists maintain that doctors sho...,What is the rationale for the patient's argume...,2,{'context': 'Patient: Pharmacists maintain tha...,True,train_1_sub1,[Lack of pharmacists' knowledge was not highli...
1,train_1,Patient: Pharmacists maintain that doctors sho...,What is the evidence that the patient's argume...,0,{'context': 'Patient: Pharmacists maintain tha...,False,train_1_sub4,[Lack of pharmacists' knowledge was not highli...
2,train_11,Biologist: A careful study of the behavior of ...,Why is the biologist's argument considered vul...,0,{'context': 'Biologist: A careful study of the...,True,train_11_sub1,[The writer refers here to focusing on who wro...
3,train_11,Biologist: A careful study of the behavior of ...,Why is the reasoning in the biologist's argume...,0,{'context': 'Biologist: A careful study of the...,False,train_11_sub2,"[Here, the author points towards conclusions o..."
4,train_10,Detective: Laser-printer drums are easily dama...,Explain why the fact that most suspicious docu...,1,{'context': 'Detective: Laser-printer drums ar...,False,train_10_sub1,[This weakens the argument because it tells th...
...,...,...,...,...,...,...,...,...
2998,train_1085,Human rights activists have urged consumers to...,Explain why the principle that it is more impo...,3,{'context': 'Human rights activists have urged...,False,train_1085_sub3,[There is no contradiction because the whole t...
2999,train_1263,PhishCo runs a number of farms in the arid pro...,Why does the fact that the government of Nufa ...,3,{'context': 'PhishCo runs a number of farms in...,False,train_1263_sub4,[Ponds on low-lying lands would make it harder...
3000,train_1341,Some government economists view their home cou...,Why does the argument's conclusion follow logi...,3,{'context': 'Some government economists view t...,True,train_1341_sub3,[Economics and the physical sciences are diffe...
3001,train_1330,"A mass of ""black water"" containing noxious org...",What is the reasoning for why the argument doe...,3,"{'context': 'A mass of ""black water"" containin...",False,train_1330_sub4,"[The level of vulnerability of the coral, olde..."


In [23]:
data

,context,question,answers,label,id_string,input_string,inference
0,Patient: Pharmacists maintain that doctors sho...,What is the rationale for the patient's argume...,[Lack of pharmacists' knowledge was not highli...,2,train_1_sub1,"Below is an instruction that describes a task,...",The answer is Option 3. The
1,Patient: Pharmacists maintain that doctors sho...,What is the evidence that the patient's argume...,[Lack of pharmacists' knowledge was not highli...,0,train_1_sub4,"Below is an instruction that describes a task,...",The answer is Option 2. The
2,Biologist: A careful study of the behavior of ...,Why is the biologist's argument considered vul...,[The writer refers here to focusing on who wro...,0,train_11_sub1,"Below is an instruction that describes a task,...",The answer is Option 1. The
3,Biologist: A careful study of the behavior of ...,Why is the reasoning in the biologist's argume...,"[Here, the author points towards conclusions o...",0,train_11_sub2,"Below is an instruction that describes a task,...",The answer is Option 3: The
4,Detective: Laser-printer drums are easily dama...,Explain why the fact that most suspicious docu...,[This weakens the argument because it tells th...,1,train_10_sub1,"Below is an instruction that describes a task,...",The answer is Option 2. The
...,...,...,...,...,...,...,...
2998,Human rights activists have urged consumers to...,Explain why the principle that it is more impo...,[There is no contradiction because the whole t...,3,train_1085_sub3,"Below is an instruction that describes a task,...",The answer is Option 3. The
2999,PhishCo runs a number of farms in the arid pro...,Why does the fact that the government of Nufa ...,[Ponds on low-lying lands would make it harder...,3,train_1263_sub4,"Below is an instruction that describes a task,...",The answer is Option 2. The
3000,Some government economists view their home cou...,Why does the argument's conclusion follow logi...,[Economics and the physical sciences are diffe...,3,train_1341_sub3,"Below is an instruction that describes a task,...","The answer is Option 1: """
3001,"A mass of ""black water"" containing noxious org...",What is the reasoning for why the argument doe...,"[The level of vulnerability of the coral, olde...",3,train_1330_sub4,"Below is an instruction that describes a task,...",The answer is Option 3. The


In [24]:
data['prediction'] = data['inference'].apply(extract_first_num_in_string)

In [25]:
data

,context,question,answers,label,id_string,input_string,inference,prediction
0,Patient: Pharmacists maintain that doctors sho...,What is the rationale for the patient's argume...,[Lack of pharmacists' knowledge was not highli...,2,train_1_sub1,"Below is an instruction that describes a task,...",The answer is Option 3. The,2
1,Patient: Pharmacists maintain that doctors sho...,What is the evidence that the patient's argume...,[Lack of pharmacists' knowledge was not highli...,0,train_1_sub4,"Below is an instruction that describes a task,...",The answer is Option 2. The,1
2,Biologist: A careful study of the behavior of ...,Why is the biologist's argument considered vul...,[The writer refers here to focusing on who wro...,0,train_11_sub1,"Below is an instruction that describes a task,...",The answer is Option 1. The,0
3,Biologist: A careful study of the behavior of ...,Why is the reasoning in the biologist's argume...,"[Here, the author points towards conclusions o...",0,train_11_sub2,"Below is an instruction that describes a task,...",The answer is Option 3: The,2
4,Detective: Laser-printer drums are easily dama...,Explain why the fact that most suspicious docu...,[This weakens the argument because it tells th...,1,train_10_sub1,"Below is an instruction that describes a task,...",The answer is Option 2. The,1
...,...,...,...,...,...,...,...,...
2998,Human rights activists have urged consumers to...,Explain why the principle that it is more impo...,[There is no contradiction because the whole t...,3,train_1085_sub3,"Below is an instruction that describes a task,...",The answer is Option 3. The,2
2999,PhishCo runs a number of farms in the arid pro...,Why does the fact that the government of Nufa ...,[Ponds on low-lying lands would make it harder...,3,train_1263_sub4,"Below is an instruction that describes a task,...",The answer is Option 2. The,1
3000,Some government economists view their home cou...,Why does the argument's conclusion follow logi...,[Economics and the physical sciences are diffe...,3,train_1341_sub3,"Below is an instruction that describes a task,...","The answer is Option 1: """,0
3001,"A mass of ""black water"" containing noxious org...",What is the reasoning for why the argument doe...,"[The level of vulnerability of the coral, olde...",3,train_1330_sub4,"Below is an instruction that describes a task,...",The answer is Option 3. The,2
